# Kaggle Global-Wheat-Detection Dataset <a class="anchor" id="top"></a>

https://www.kaggle.com/c/global-wheat-detection

## Table of Content
* [Visualization Helpers](#visu_helpers)
* [Data Helpers](#data_helpers)
* [Prepare Data](#prepare_data)
* [Visualize Data](#visualize_data)
* [Generate Model](#generate_model)
* [Train Model](#train_model)
* [Show Results)(#show_results)

### Useful Notebooks


In [ ]:
# imports
import numpy as np
import pandas as pd

# Visualization Helpers <a class="anchor" id="visu_helpers"></a> 
[got to top](#to)


In [ ]:
"""@package visu_helpers

  @brief Helper functions to visualize data (e.g. bounding boxes)
  
  @author Maximilian Harr <maximilian.harr@gmail.com>
  @date 01.06.2020

  @bug
  @warning
  @todo
 
"""

def plot_boundingbox(data_box, image_str):
    """
    plot_boundingbox Plot image with bounding boxes
        @param data_box: xx
        @param_image_str: xx
        @return: xx
    """
    
    # Check input parameter
    if not type(data_box) == str and not type(image_str) == :
        raise ValueError('Wrong datatype provided')
        
    # Get image and it's bounding boxes
    image = io.imread(image_str)
    bbox_all = get_img_bbox(data_box[data_box.image_id == image_str])
    
    # Plot data
    fig, ax = plt.subplots(1)
    ax.imshow(image)
    
    for bbox in bbox_all.reshape(-1,4):
        rect = patches.Rectangle((bbox[0], bbox[1]), bbox[2], bbox[3],linewidth=1,edgecolor='r',facecolor='none')                    
        ax.add_patch(rect)
    
    plt.show()

In [ ]:
"""@package test_visu_helpers

  @brief Unittest for visu helpers
  
  @author Maximilian Harr <maximilian.harr@gmail.com>
  @date 01.06.2020

  @bug
  @warning
  @todo
 
"""

# IMPORTS ########################################################################################
import unittest
import math
import numpy as np

# CLASSES ########################################################################################

class TestVisuHelpers(unittest.TestCase):

    def test_get_array_from_string(self):
        array = get_array_from_string("[1.2, 2.3, 3.4]")
        
        self.assertEqual( array[0], 1.2 )
        self.assertEqual( array[1], 2.3 )
        self.assertEqual( array[2], 3.4 )

# FUNCTIONS ######################################################################################


# MAIN ###########################################################################################
if __name__ == '__main__':
    #unittest.main()
    unittest.main(argv=[''], verbosity=2, exit=False)


# Data Helpers <a class="anchor" id="data_helpers"></a>
[got to top](#top)

In [ ]:
"""@package data_helpers

  @brief Helper functions to convert text data (e.g. csv etc)
  
  @author Maximilian Harr <maximilian.harr@gmail.com>
  @date 29.05.2020

  @bug
  @warning
  @todo
 
"""

## IMPORTS #######################################################################################
import numpy as np
import scipy.io
from skimage import io, exposure
import matplotlib.pyplot as plt

## CLASSES #######################################################################################

## FUNCTIONS #####################################################################################

def get_array_from_string(datastring: str) -> np.array([]):
    """
    get_array_from_string Reads array from string and returns it as np.array
        @param array_str: string of arrays, e.g. "[1, 2, 3]"
        @return: np.array
    """

    # Check input parameter
    if not type(datastring) == str:
        raise ValueError('Wrong datatype provided')
    
    # Extract paranthesis from string
    datastring = datastring.replace("[", "")
    datastring = datastring.replace("]", "")

    data_array = np.array([])
    data_array = datastring.split(",")
    
    # Convert to float
    data_array = [float(e) for e in data_array]
    
    return data_array

def get_img_bbox(data):
    """
    get_img_bbox Get bbox arrays from (cropped) pandas (train.csv)
        @param data: xxx
        @return: np.array
    """
    
    # Check input parameter
    if not type(data) == str:
        raise ValueError('Wrong datatype provided')
        
    bbox_all = np.array([])
    
    # Iterate boxes
    for index, row in data.iterrows():
        bbox_f = get_array_from_string( row['bbox'] )
        bbox_all = np.concatenate([bbox_all, bbox_f])

    return bbox_all

In [ ]:
"""@package test_data_helpers

  @brief Unittest for data helpers
  
  @author Maximilian Harr <maximilian.harr@gmail.com>
  @date 29.05.2020

  @bug
  @warning
  @todo
 
"""

# IMPORTS ########################################################################################
import unittest
import math
import numpy as np

# CLASSES ########################################################################################

class TestDataHelpers(unittest.TestCase):

    def test_get_array_from_string(self):
        array = get_array_from_string("[1.2, 2.3, 3.4]")
        
        self.assertEqual( array[0], 1.2 )
        self.assertEqual( array[1], 2.3 )
        self.assertEqual( array[2], 3.4 )

# FUNCTIONS ######################################################################################


# MAIN ###########################################################################################
if __name__ == '__main__':
    #unittest.main()
    unittest.main(argv=[''], verbosity=2, exit=False)


# Prepare Data <a class="anchor" id="prepare_data"></a>
[got to top](#top)

# Visualize Data <a class="anchor" id="visualize_data"></a>
[got to top](#top)

In [ ]:
# 

# Generate Model <a class="anchor" id="generate_model"></a>
[got to top](#top)

# Train Model <a class="anchor" id="train_model"></a>
[got to top](#top)

# Show Results <a class="anchor" id="show_results"></a>
[got to top](#top)

In [ ]:
array = "1"
array = wheat_helpers.get_array_from_string("[1, 2, 3]")
print(array[1])

In [ ]:
kaggledata_root = "../../res/data" # Leave empty for upload on kaggle

In [ ]:
#read csv and show head of csv
data_box = pd.read_csv(kaggledata_root + "/kaggle/input/global-wheat-detection/train.csv")
data_box.head()

wheat_helpers.plot_boundingbox(data_box, kaggledata_root + '/kaggle/input/global-wheat-detection/train/b6ab77fd7.jpg')

In [ ]:
# Display sample image
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
img = mpimg.imread(kaggledata_root + "/kaggle/input/global-wheat-detection/train/b6ab77fd7.jpg")
plt.imshow(img)